<a href="https://colab.research.google.com/github/cadejs/201-final/blob/master/Task_1_Data_Munging_on_Monthly_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive 
drive.mount('/content/gdrive')
from google.colab import auth
from google.colab import runtime
auth.authenticate_user()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


This notebook will:


1.   Join Months into Quaters (done and stored)
2.   Join Quaters into Years (reduction to 50 random sites needed first)
3.   Reduce years & quaters in size by selecting 50 random sites (rather than a more methodical way)
4.   Join NOAA Dataset to augment weather data (code to be written not exectued)
5.   Join RBSA Data (code to be written not executed
6.   Reduce the number of rows by aggregating up to the site level (done)






In [ ]:
#join months into quaters and quaters into years, currenlty joining full quaters crashes ram so reduction is needed
def join_neaa_data(*dfs):
  return pd.concat(dfs, ignore_index=True)

In [ ]:
#group by site id, time, circuit category and state(maybe remove) then create a column for sum of power for each circuit type and a total power column then aggregate again
def reduce_points(df,full_reduction):
  #inital groupby
  df_agg = df.groupby(["ee_site_id",'circuit_category','min_t']).agg({"circuit_label_type_desc": "unique","power": "sum","apparent_power": "sum","station_id": "unique",
                                                                                   "temp_f": "mean","thd": "mean"}).reset_index()
  df_agg.assign(Non_Forced_Air_Plus=0, HP=0,Forced_Air=0,AC = 0) 
  df_agg['Non_Forced_Air_Plus'] = np.where(df_agg['circuit_category'] == 'Non-Forced Air Plus',df_agg['power'],0)
  df_agg['HP'] = np.where(df_agg['circuit_category'] == 'HP',df_agg['power'],0)
  df_agg['Forced_Air'] = np.where(df_agg['circuit_category'] == 'Forced Air',df_agg['power'],0)
  df_agg['AC'] = np.where(df_agg['circuit_category'] == 'AC',df_agg['power'],0)
  df_agg['circuit_label_type_desc'] = df_agg['circuit_label_type_desc'].apply(lambda x: tuple(x))
  df_agg['circuit_category'] = df_agg['circuit_category'].apply(lambda x: tuple(x))
  df_agg['station_id'] = df_agg['station_id'].apply(lambda x: tuple(x))
  df_agg_equipment_level = df_agg
  #second group by to furhter reduce file size
  if full_reduction:
    df_agg_site_level = df_agg.groupby(["ee_site_id",'min_t']).agg({"circuit_label_type_desc": "unique","circuit_category": "unique","power": "sum","apparent_power": "sum","station_id": "unique",
                                                                                   "temp_f": "mean","thd": "mean","AC":"sum","HP":"sum","Forced_Air":"sum","Non_Forced_Air_Plus":"sum"}).reset_index()
    #can make the power as a % if needed      
    return df_agg_equipment_level, df_agg_site_level
  else:
    return df_agg_equipment_level, None


In [ ]:
def remove_columns(path):
    for file in os.listdir(path):
        if file.endswith(".parquet"):
            df = pd.read_parquet(path + file)
            print(df.columns)
            df = df.drop(['circuit_label_type_desc','circuit_category','station_id'],axis=1)
            df.to_parquet(path + file)
path = '/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/'
remove_columns(path)

path = '/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/'
remove_columns(path)


Index(['ee_site_id', 'circuit_category', 'min_t', 'circuit_label_type_desc',
       'power', 'apparent_power', 'station_id', 'temp_f', 'thd',
       'Non_Forced_Air_Plus', 'HP', 'Forced_Air', 'AC'],
      dtype='object')
Index(['ee_site_id', 'circuit_category', 'min_t', 'circuit_label_type_desc',
       'power', 'apparent_power', 'station_id', 'temp_f', 'thd',
       'Non_Forced_Air_Plus', 'HP', 'Forced_Air', 'AC'],
      dtype='object')
Index(['ee_site_id', 'circuit_category', 'min_t', 'circuit_label_type_desc',
       'power', 'apparent_power', 'station_id', 'temp_f', 'thd',
       'Non_Forced_Air_Plus', 'HP', 'Forced_Air', 'AC'],
      dtype='object')
Index(['ee_site_id', 'circuit_category', 'min_t', 'circuit_label_type_desc',
       'power', 'apparent_power', 'station_id', 'temp_f', 'thd',
       'Non_Forced_Air_Plus', 'HP', 'Forced_Air', 'AC'],
      dtype='object')
Index(['ee_site_id', 'circuit_category', 'min_t', 'circuit_label_type_desc',
       'power', 'apparent_power', 'stat

In [ ]:
test_p = pd.read_parquet('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/19_q1_equipment.parquet')
test_p.head(5)
runtime.unassign()

In [ ]:
#file size is to big to join, generate 50 random sites to join (4 times _,a,b,c), saved them in a list and as a csv 
random_site_50 = pd.read_csv('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/neaa_end_use/sites_random_50.csv')
# site = pd.read_csv('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/neaa_end_use/sites.csv')
# site_list_50 = site.sample(n=50)
site_list_50 = random_site_50['ee_site_id'].to_list()

In [ ]:
#tryin to do site and equipment yearly values here, will edit this to do that once reduction occurs

#yearly read in
# q1_19_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/19_q1_equipment.parquet')
# q2_19_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/19_q2_equipment.parquet')
# q3_19_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/19_q3_equipment.parquet')
# q4_19_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/19_q4_equipment.parquet')

#yearly agg
# equipment_2019 = join_neaa_data(q1_19_equipment,q2_19_equipment,q3_19_equipment,q4_19_equipment)
# equipment_2019.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/2019_equipment.parquet')
# del equipment_2019, q1_19_equipment,q2_19_equipment,q3_19_equipment,q4_19_equipment

# q1_19_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/19_q1_site.parquet').query('ee_site_id in @site_list_50')
# print(q1_19_site.memory_usage(index=True).sum()/1000000000)
# q2_19_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/19_q2_site.parquet').query('ee_site_id in @site_list_50')
# print(q2_19_site.memory_usage(index=True).sum()/1000000000)
# site_2019_1 = join_neaa_data(q1_19_site,q2_19_site)
# site_2019_1.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/year/2019_1h_site_random_50.parquet')

# del q1_19_site,q2_19_site, site_2019_1
# q3_19_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/19_q3_site.parquet').query('ee_site_id in @site_list_50')
# print(q3_19_site.memory_usage(index=True).sum()/1000000000)
# q4_19_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/19_q4_site.parquet').query('ee_site_id in @site_list_50')
# print(q4_19_site.memory_usage(index=True).sum()/1000000000)
# site_2019_2 = join_neaa_data(q3_19_site,q4_19_site)
# site_2019_2.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/year/2019_2h_site_random_50.parquet')
# del q3_19_site,q4_19_site, site_2019_2
# site_2019 = join_neaa_data(site_2019_1, site_2019_2)
# site_2019.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/2020_site.parquet')
# del site_2019, q1_19_site,q2_19_site,q3_19_site,q4_19_site

# #yearly read in
# q1_20_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/20_q1_equipment.parquet')
# q2_20_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/20_q2_equipment.parquet')
# equipment_2020_1 = join_neaa_data(q1_20_equipment,q2_20_equipment)
# equipment_2020_1.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/2020_equipment_1.parquet')
# del equipment_2020_1, q1_20_equipment,q2_20_equipment

# q3_20_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/20_q3_equipment.parquet')
# q4_20_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/20_q4_equipment.parquet')

# #yearly agg
# equipment_2020 = join_neaa_data(q1_20_equipment,q2_20_equipment,q3_20_equipment,q4_20_equipment)
# equipment_2020.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/2020_equipment.parquet')
# del equipment_2020, q1_20_equipment,q2_20_equipment,q3_20_equipment,q4_20_equipment

# q1_20_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/20_q1_site.parquet')
# q2_20_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/20_q2_site.parquet')
# q3_20_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/20_q3_site.parquet')
# q4_20_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/20_q4_site.parquet')
# site_2020 = join_neaa_data(q1_20_site,q2_20_site,q3_20_site,q4_20_site)
# site_2020.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/2020_site.parquet')

# del site_2020, q1_20_site,q2_20_site,q3_20_site,q4_20_site

# #yearly read in
# q1_21_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/21_q1_equipment.parquet')
# q2_21_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/21_q2_equipment.parquet')
# q3_21_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/21_q3_equipment.parquet')
# q4_21_equipment = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/21_q4_equipment.parquet')


# #yearly agg
# equipment_2021 = join_neaa_data(q1_21_equipment,q2_21_equipment,q3_21_equipment,q4_21_equipment)
# equipment_2021.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/2021_equipment.parquet')
# del equipment_2021, q1_21_equipment,q2_21_equipment,q3_21_equipment,q4_21_equipment
# q1_21_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/21_q1_site.parquet')
# q2_21_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/21_q2_site.parquet')
# q3_21_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/21_q3_site.parquet')
# q4_21_site = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/21_q4_site.parquet')
# site_2021 = join_neaa_data(q1_21_site,q2_21_site,q3_21_site,q4_21_site)

# site_2021.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/2020_site.parquet')
# del site_2021, q1_21_equipment,q2_21_equipment,q3_21_equipment,q4_21_equipment

# runtime.unassign()

In [ ]:
#join NOAA data - dev already done some

In [ ]:
# Join RBSA Dataset

# join the site, RBSA and CBSA
cbsa_site = pd.read_csv(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/RBSA & CBSA/CBSA-4-Data-Files/site_details-facility.csv')[['facility_id',	'site_id','facility_type','building_area_conditioned','year_constructed','heating_fuel','cooling_fuel','res_square_feet','energy_sources','site_reference_number','primary_building_type_site','region','state','building_total_floor_area','building_conditioned_floor_area']]
rbsa_site = pd.read_csv(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/RBSA & CBSA/RBSA-II-Combined-Database/SiteOneLine.csv')[['CK_SiteID','Building Category','Building Type',	'Region',	'County',	'City',	'State','Zip','Electric Utility','Strata Territory','Vintage','Conditioned Area','Primary Heating System', 'Primary Heating Fuel',	'Primary Cooling System']]


# Indicate data source
cbsa_site['res_or_comm'] = 'commercial'
rbsa_site['res_or_comm'] = 'residential'

#add site_id in correct format
def format_number(number):
    return "SITE_" + str(number).zfill(5)
cbsa_site['site_id_universal'] = cbsa_site['site_id'].apply(format_number)

#cbsa heating and cooling breakdown
cbsa_site.assign(cool_type='', heat_type='') #natural_gas_plus to handle the odd fuel sources
cbsa_site.loc[cbsa_site["cooling_fuel"] == 'Electricity', "cool_type"] = 'Electricty'
cbsa_site.loc[cbsa_site["cooling_fuel"] != 'Electricity', "cool_type"] = 'None'
cbsa_site.loc[cbsa_site["heating_fuel"] == 'Electricity', "heat_type"] = 'Electricty'
cbsa_site.loc[cbsa_site["heating_fuel"] != 'Electricity', "heat_type"] = 'natural_gas_plus'

#rbsa heating and cooling breakdown
rbsa_site.assign(cool_type='', heat_type='',cooling_equipment='',heating_equipment = '') 
rbsa_site.loc[rbsa_site["Primary Cooling System"].isnull(), "cool_type"] = 'None'
rbsa_site.loc[~rbsa_site["Primary Cooling System"].isnull(), "cool_type"] = 'Electricty'
rbsa_site.loc[rbsa_site["Primary Cooling System"].isnull(), "cooling_equipment"] = 'None'
rbsa_site.loc[rbsa_site["Primary Cooling System"].notnull() & rbsa_site["Primary Cooling System"].str.contains("AC|Packaged Unit"), "cooling_equipment"] = 'AC'
rbsa_site.loc[rbsa_site["Primary Cooling System"].notnull() & rbsa_site["Primary Cooling System"].str.contains(r"\bHeat Pump\b|\bHP\b"), "cooling_equipment"] = 'HP'
rbsa_site.loc[rbsa_site["Primary Cooling System"].notnull() & rbsa_site["Primary Cooling System"].str.contains(r"Unknown|Evaporative Cooling"), "cooling_equipment"] = 'Unknown'


#heating grouping
rbsa_site.loc[rbsa_site["Primary Heating Fuel"] == 'Electric', "heat_type"] = 'Electricty'
rbsa_site.loc[rbsa_site["Primary Heating Fuel"] != 'Electric', "heat_type"] = 'natural_gas_plus'
rbsa_site.loc[rbsa_site["Primary Heating System"].notnull() & rbsa_site["Primary Heating System"].str.contains(r"\bHeat Pump\b|HP"), "heating_equipment"] = 'HP'
rbsa_site.loc[rbsa_site["Primary Heating System"].notnull() & rbsa_site["Primary Heating System"].str.contains(r"Furnace|Zonal Heat|Packaged Unit"), "heating_equipment"] = 'Forced Air'
rbsa_site.loc[rbsa_site["Primary Heating System"].notnull() & rbsa_site["Primary Heating System"].str.contains(r"Electric Baseboard|Stove/Fireplace|Boiler|Radiant Heat|Storage Water Heater"), "heating_equipment"] = 'Non-Forced Air Plus'
rbsa_site.loc[rbsa_site["Primary Heating System"].notnull() & rbsa_site["Primary Heating System"].str.contains(r"Unknown"), "heating_equipment"] = 'Unknown'

In [ ]:
# #2018
# q3_18 = pd.read_parquet('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q3_18.parquet')
# q3_18_equipment, q3_18_site = reduce_points(q3_18,True)
# del q3_18
# q3_18_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/18_q3_equipment.parquet')
# q3_18_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/18_q3_site.parquet')
# del q3_18_equipment, q3_18_site

# q4_18 = pd.read_parquet('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q4_18.parquet')
# q4_18_equipment, q4_18_site = reduce_points(q4_18,True)
# del q4_18

# q4_18_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/18_q4_equipment.parquet')
# q4_18_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/18_q4_site.parquet')
# del q4_18_equipment, q4_18_site

#2019
# q1_19 = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q1_19.parquet')
# q1_19_equipment, q1_19_site = reduce_points(q1_19,True)
# del q1_19
# q1_19_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/19_q1_equipment.parquet')
# q1_19_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/19_q1_site.parquet')
# del q1_19_equipment, q1_19_site

# q2_19 = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q2_19.parquet')
# q2_19_equipment, q2_19_site = reduce_points(q2_19,True)
# del q2_19
# q2_19_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/19_q2_equipment.parquet')
# q2_19_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/19_q2_site.parquet')
# del q2_19_equipment, q2_19_site

# q3_19 = pd.read_parquet('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q3_19.parquet')
# q3_19_equipment, q3_19_site = reduce_points(q3_19,True)
# del q3_19
# q3_19_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/19_q3_equipment.parquet')
# q3_19_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/19_q3_site.parquet')
# del q3_19_equipment, q3_19_site

# q4_19 = pd.read_parquet('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q4_19.parquet')
# q4_19_equipment, q4_19_site = reduce_points(q4_19,True)
# del q4_19

# q4_19_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/19_q4_equipment.parquet')
# q4_19_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/19_q4_site.parquet')
# del q4_19_equipment, q4_19_site

#2020
# q1_20 = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q1_20.parquet')
# q1_20_equipment, q1_20_site = reduce_points(q1_20,True)
# del q1_20
# q1_20_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/20_q1_equipment.parquet')
# q1_20_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/20_q1_site.parquet')
# del q1_20_equipment, q1_20_site

# q2_20 = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q2_20.parquet')
# q2_20_equipment, q2_20_site = reduce_points(q2_20,True)
# del q2_20
# q2_20_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/20_q2_equipment.parquet')
# q2_20_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/20_q2_site.parquet')
# del q2_20_equipment, q2_20_site

# q3_20 = pd.read_parquet('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q3_20.parquet')
# q3_20_equipment, q3_20_site = reduce_points(q3_20,True)
# del q3_20
# q3_20_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/20_q3_equipment.parquet')
# q3_20_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/20_q3_site.parquet')
# del q3_20_equipment, q3_20_site

# q4_20 = pd.read_parquet('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q4_20.parquet')
# q4_20_equipment, q4_20_site = reduce_points(q4_20,True)
# del q4_20

# q4_20_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/20_q4_equipment.parquet')
# q4_20_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/20_q4_site.parquet')
# del q4_20_equipment, q4_20_site


# #2021
# q1_21 = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q1_21.parquet')
# q1_21_equipment, q1_21_site = reduce_points(q1_21,True)
# del q1_21
# q1_21_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/21_q1_equipment.parquet')
# q1_21_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/21_q1_site.parquet')
# del q1_21_equipment, q1_21_site

# q2_21 = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q2_21.parquet')
# q2_21_equipment, q2_21_site = reduce_points(q2_21,True)
# del q2_21
# q2_21_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/21_q2_equipment.parquet')
# q2_21_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/21_q2_site.parquet')
# del q2_21_equipment, q2_21_site

# q3_21 = pd.read_parquet('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q3_21.parquet')
# q3_21_equipment, q3_21_site = reduce_points(q3_21,True)
# del q3_21
# q3_21_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/21_q3_equipment.parquet')
# q3_21_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/21_q3_site.parquet')
# del q3_21_equipment, q3_21_site

# q4_21 = pd.read_parquet('/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q4_21.parquet')
# q4_21_equipment, q4_21_site = reduce_points(q4_21,True)
# del q4_21

# q4_21_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/21_q4_equipment.parquet')
# q4_21_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/21_q4_site.parquet')
# del q4_21_equipment, q4_21_site

# #2022
# q1_22 = pd.read_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/q1_22.parquet')
# q1_22_equipment, q1_22_site = reduce_points(q1_22,True)
# del q1_22
# q1_22_equipment.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/equipment/22_q1_equipment.parquet')
# q1_22_site.to_parquet(r'/content/gdrive/MyDrive/Digitizing Utilities Challenge/NEAA_end_use_metering/Processed_data/site/22_q1_site.parquet')
# del q1_22_equipment, q1_22_site
